## Reconstruct from original text
> this file contains every field from the azure transformations,
> you can restart from here if you want to construct new features

In [3]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

In [12]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [2]:
#sc.stop()
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '6g')
#SparkContext.setSystemProperty('spark.rdd.compress', True)
SparkContext.setSystemProperty('spark.driver.memory', '16g')
sc = SparkContext(master="local[4]")
print(sc)


<SparkContext master=local[4] appName=pyspark-shell>


In [13]:
path='/home/ucsddse230/work/final_project/azure/Tokens_vectors_text/final_tokens_with_text.parquet'

In [14]:
spark = SparkSession.builder.getOrCreate()

### this file contains everything that happend before vectorization
#### includung ** original text **

In [15]:
processed_azure_token_original_text= spark.read.parquet(path)

In [16]:
processed_azure_token_original_text.dtypes

[('HADM_ID', 'int'),
 ('SUBJECT_ID', 'int'),
 ('ADMITTIME', 'timestamp'),
 ('DISCHTIME', 'timestamp'),
 ('DEATHTIME', 'timestamp'),
 ('ADMISSION_TYPE', 'string'),
 ('NEXT_ADMISSION', 'timestamp'),
 ('NEXT_ADMISSION_TYPE', 'string'),
 ('days_next_admit', 'int'),
 ('CATEGORY', 'string'),
 ('TEXT', 'string'),
 ('row_number', 'int'),
 ('label', 'int'),
 ('TEXT_clean', 'string'),
 ('tokens', 'array<string>'),
 ('tokens_clean', 'array<string>')]

In [47]:
# processed_azure_token_original_text[["tokens"]].take(1)

## prepare Training test and validation data

In [20]:
#grab relavant fields
subset_for_features=processed_azure_token_original_text.select('HADM_ID', 'SUBJECT_ID', 'label','tokens_clean' )

In [24]:
seed=6250

In [25]:
splits=subset_for_features.randomSplit([.7, 0.2,0.1], seed)

In [27]:
print("Training obs {} , validation obs {} testing obs {}".format(splits[0].count(),splits[1].count(),splits[2].count()))

Training obs 35598 , validation obs 10330 testing obs 5185


In [28]:
#extract training testing and validation data
training=splits[0]
validation=splits[1]
testing=splits[2]

In [30]:
token_vector = CountVectorizer(inputCol="tokens_clean", outputCol="features")
vector_model=token_vector.fit(training)
# .transform(final_data).cache()

### Model Terms

In [32]:
vector_model.vocabulary

['patient',
 'blood',
 'discharge',
 'daily',
 'sig',
 'one',
 'left',
 'history',
 'right',
 'pain',
 'date',
 'normal',
 'chest',
 'well',
 'status',
 'home',
 'capsule',
 'stable',
 'follow',
 'course',
 'care',
 'disease',
 'days',
 'two',
 'artery',
 'wbc',
 'aortic',
 'neg',
 'valve',
 'negative',
 'medical',
 'glucose',
 'past',
 'per',
 'post',
 'exam',
 'cardiac',
 'acute',
 'mild',
 'present',
 'pulmonary',
 'family',
 'ventricular',
 'physical',
 'without',
 'heart',
 'edema',
 'rbc',
 'weeks',
 'hgb',
 'procedure',
 'surgical',
 'due',
 'urine',
 'surgery',
 'pressure',
 'condition',
 'evidence',
 'mch',
 'small',
 'renal',
 'delayed',
 'lower',
 'non',
 'mitral',
 'hypertension',
 'tube',
 'systolic',
 'chronic',
 'fluid',
 'soft',
 'failure',
 'coronary',
 'head',
 'abdominal',
 'illness',
 'social',
 'aspirin',
 'calcium',
 'atrial',
 'rate',
 'old',
 'abdomen',
 'effusion',
 'bowel',
 'respiratory',
 'bilateral',
 'changes',
 'wall',
 'secondary',
 'low',
 'floor',
 'th

In [31]:
training_features = vector_model.transform(training).cache()
validation_features =  vector_model.transform(validation).cache()
testing_features = vector_model.transform(testing).cache()

In [40]:
#notice the same length features
print('------------------Training----------------------------')
print(training_features[["features"]].take(1))
print('------------------------------------------------------')
print('-------------Validation-------------------------------')
print(validation_features[["features"]].take(1))
print('--------------Testing---------------------------------')
print('------------------------------------------------------')
print(testing_features[["features"]].take(1))

------------------Training----------------------------
[Row(features=SparseVector(4456, {0: 5.0, 1: 2.0, 2: 7.0, 3: 1.0, 4: 5.0, 5: 4.0, 6: 8.0, 7: 6.0, 8: 1.0, 9: 13.0, 10: 3.0, 11: 1.0, 12: 1.0, 13: 2.0, 14: 1.0, 15: 2.0, 16: 3.0, 17: 2.0, 18: 4.0, 19: 2.0, 20: 1.0, 23: 1.0, 25: 1.0, 30: 1.0, 31: 1.0, 32: 2.0, 34: 1.0, 35: 2.0, 39: 1.0, 41: 2.0, 43: 1.0, 44: 1.0, 47: 1.0, 48: 1.0, 49: 1.0, 50: 1.0, 51: 2.0, 52: 1.0, 53: 1.0, 54: 1.0, 56: 2.0, 57: 1.0, 58: 1.0, 60: 4.0, 62: 1.0, 66: 1.0, 70: 1.0, 75: 1.0, 76: 1.0, 77: 1.0, 81: 1.0, 85: 1.0, 91: 1.0, 98: 1.0, 99: 1.0, 101: 1.0, 102: 1.0, 106: 1.0, 107: 1.0, 110: 1.0, 113: 1.0, 116: 1.0, 119: 1.0, 122: 2.0, 131: 1.0, 142: 2.0, 144: 1.0, 147: 3.0, 164: 1.0, 167: 2.0, 171: 1.0, 177: 1.0, 180: 1.0, 185: 2.0, 189: 1.0, 190: 1.0, 192: 1.0, 196: 2.0, 197: 2.0, 198: 1.0, 207: 1.0, 208: 1.0, 212: 1.0, 219: 1.0, 230: 1.0, 234: 1.0, 241: 2.0, 246: 1.0, 247: 1.0, 249: 1.0, 258: 1.0, 276: 1.0, 279: 1.0, 301: 2.0, 327: 1.0, 343: 2.0, 351: 1.0, 365: 

In [43]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

### save data sets as parquet

In [46]:
training_features.write.parquet("training_features.parquet")
validation_features.write.parquet("validation_features.parquet")
testing_features.write.parquet("testing_features.parquet")

In [49]:
loaded=spark.read.parquet("training_features.parquet")